#### Init

In [22]:
# Connect to server
import hackathon_linc as lh
import math

lh.init('5d5249da-7e7e-438b-a392-692f71364fd0')



    Welcome to the LINC Hackathon! Your token is now saved in the Console. 
    That means you don't need to carry that out when using the other functions
    as long as you don't close your console. 
    
    This function is only to be used once to authenticate your token.
    
    Happy coding!

    


#### Buy index at start

In [ ]:
import hackathon_linc as lh
import math

starting_budget = lh.get_balance()

percentage = float(input("Enter the percentage of the total budget to invest in INDEX1 (e.g., 10 for 10%): "))
investment_amount = starting_budget * (percentage / 100)
price_data = lh.get_current_price("INDEX1")
print("Price data returned by API:", price_data)

print()

# Extract the ask price for buying from the returned structure
current_price = price_data['data'][0]['askMedian']

# Calculate the number of shares to buy (using floor division to get an integer number)
shares_to_buy = int(investment_amount // current_price)

print(f"Attempting to buy {shares_to_buy} shares of INDEX1 at {current_price} per share.")
print(f"This will invest approximately {shares_to_buy * current_price} out of your allocated {investment_amount}.")

# Place the buy order; here, days_to_cancel is set to 1 (adjust as needed)
buy_response = lh.buy("INDEX1", shares_to_buy)
print("Buy order response:", buy_response)


Price data returned by API: {'data': [{'askMedian': 249.82, 'askVolume': 1000.0, 'bidMedian': 234.61, 'bidVolume': 300.0, 'gmtTime': '2004-01-02 07:00:00+00:00', 'spreadMedian': 15.20948200000001, 'symbol': 'INDEX1'}]}

Attempting to buy 40 shares of INDEX1 at 249.82 per share.
This will invest approximately 9992.8 out of your allocated 10018.611.
Buy order response: {'amount': 40, 'created_at': '2004-11-03T12:36:00', 'ended_at': None, 'id': '80a3bb94-bcc6-4d92-a4a0-69398abb1bf1', 'order_status': 'pending', 'order_type': 'buy', 'price': 1.0, 'security_id': '6ee42866-e27b-41ec-b2cd-ae42f403bab4', 'team_id': '3ddb7410-231a-497c-bcbc-3b4bee175f1f'}


#### Sell all!

In [23]:
def sell_all():
    port = lh.get_portfolio()
    for key, quantity in port.items():
        lh.sell(key, quantity)
sell_all()
portfolio = lh.get_portfolio()
print(portfolio)

{'STOCK1': 0, 'STOCK10': 0, 'STOCK12': 0, 'STOCK13': 0, 'STOCK14': 0, 'STOCK16': 0, 'STOCK17': 0, 'STOCK2': 0, 'STOCK21': 0, 'STOCK3': 0, 'STOCK5': 0, 'STOCK6': 0, 'STOCK9': 0}


#### Cancel all orders

In [19]:
# Retrieve all pending orders
pending_orders = lh.get_pending_orders()
print("Pending orders:", pending_orders)

# Iterate over pending orders and cancel each one
for order in pending_orders:
    order_id = order.get("order_id")
    ticker = order.get("ticker")
    if order_id and ticker:
        # Cancel the order using both order_id and ticker
        cancel_response = lh.cancel(order_id=order_id, ticker=ticker)
        print(f"Cancelled order {order_id} for ticker {ticker}: {cancel_response}")
    else:
        print(f"Skipping order with insufficient data: {order}")

Pending orders: []


##### Sell individual stock

In [28]:
selection = input("Enter your selection (0 or 1-21, excluding 15): ").strip()

# Determine the ticker symbol based on selection
if selection == "0":
    ticker = "INDEX1"
else:
    try:
        num = int(selection)
        if num < 1 or num > 21 or num == 15:
            print("Invalid selection. Please choose 0 or a number between 1 and 21 (excluding 15).")
            raise SystemExit
        ticker = f"STOCK{num}"
    except ValueError:
        print("Invalid input. Please enter a numeric value.")
        raise SystemExit

# Ask the user for the number of shares to sell or to sell all shares
shares_input = input("Enter the number of shares you wish to sell (or type 'all' to sell your entire holding): ").strip()

if shares_input.lower() == "all":
    # Retrieve the current portfolio holdings
    portfolio = lh.get_portfolio()
    print("Current portfolio holdings:", portfolio)
    # Get the number of shares held for the chosen ticker
    shares_to_sell = portfolio.get(ticker, 0)
    if shares_to_sell == 0:
        print(f"You do not hold any shares of {ticker} to sell.")
        raise SystemExit
else:
    try:
        shares_to_sell = int(shares_input)
        if shares_to_sell <= 0:
            print("The number of shares must be a positive integer.")
            raise SystemExit
    except ValueError:
        print("Invalid input. Please enter a positive integer or 'all'.")
        raise SystemExit

# Retrieve the current price for the selected ticker
price_data = lh.get_current_price(ticker)
print(f"Price data for {ticker}:", price_data)

# Extract the bid price from the returned structure for selling
if isinstance(price_data, dict) and "data" in price_data:
    current_price = price_data["data"][0]["bidMedian"]
else:
    current_price = price_data

print(f"Selling {shares_to_sell} shares of {ticker} at {current_price} per share.")
print(f"Estimated total revenue: {shares_to_sell * current_price}")

# Place the sell order; using 1 day as the default for days_to_cancel
sell_response = lh.sell(ticker, shares_to_sell)
print("Sell order response:", sell_response)

Price data for STOCK1: {'data': [{'askMedian': 278.3, 'askVolume': 5503.0, 'bidMedian': 277.98, 'bidVolume': 3340.0, 'gmtTime': '2005-02-02 15:00:00+00:00', 'spreadMedian': 0.3236059999999816, 'symbol': 'STOCK1'}]}
Selling 3 shares of STOCK1 at 277.98 per share.
Estimated total revenue: 833.94
Sell order response: {'message': 'Not enough stock to sell'}
